In [ ]:
import pandas as pd

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving Data_for_UCI_named.csv to Data_for_UCI_named.csv


In [ ]:
import io
df = pd.read_csv(io.BytesIO(uploaded['Data_for_UCI_named.csv']))

In [ ]:
import numpy as np
df.head()

,tau1,tau2,tau3,tau4,p1,p2,p3,p4,g1,g2,g3,g4,stab,stabf
0,2.959060,3.079885,8.381025,9.780754,3.763085,-0.782604,-1.257395,-1.723086,0.650456,0.859578,0.887445,0.958034,0.055347,unstable
1,9.304097,4.902524,3.047541,1.369357,5.067812,-1.940058,-1.872742,-1.255012,0.413441,0.862414,0.562139,0.781760,-0.005957,stable
2,8.971707,8.848428,3.046479,1.214518,3.405158,-1.207456,-1.277210,-0.920492,0.163041,0.766689,0.839444,0.109853,0.003471,unstable
3,0.716415,7.669600,4.486641,2.340563,3.963791,-1.027473,-1.938944,-0.997374,0.446209,0.976744,0.929381,0.362718,0.028871,unstable
4,3.134112,7.608772,4.943759,9.857573,3.525811,-1.125531,-1.845975,-0.554305,0.797110,0.455450,0.656947,0.820923,0.049860,unstable


In [ ]:
df.isna().sum()

tau1     0
tau2     0
tau3     0
tau4     0
p1       0
p2       0
p3       0
p4       0
g1       0
g2       0
g3       0
g4       0
stab     0
stabf    0
dtype: int64

In [ ]:
df['stabf'].value_counts()

unstable    6380
stable      3620
Name: stabf, dtype: int64

In [ ]:
df=df.drop(columns=['stab'])

In [ ]:
x= df.drop(columns='stabf')
y= df['stabf']

In [ ]:
#split data into training set and test set
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test= train_test_split(x,y, test_size=0.2, random_state=1)

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler= StandardScaler()
x_train= scaler.fit_transform(x_train)

In [ ]:
x_test= x_test.reset_index(drop=True)
x_test= scaler.transform(x_test)

In [ ]:
#import Random forest
from sklearn.ensemble import RandomForestClassifier
clf= RandomForestClassifier(random_state=1)
clf.fit(x_train, y_train)
new_prediction= clf.predict(x_test)

In [ ]:
from sklearn.ensemble import ExtraTreesClassifier
tree= ExtraTreesClassifier(random_state=1)
tree.fit(x_train, y_train)
new_prediction2= tree.predict(x_test)

In [ ]:
import xgboost as xgb
xg_clf= xgb.XGBClassifier(random_state=1)
xg_clf.fit(x_train, y_train)
new_prediction3= xg_clf.predict(x_test)

In [ ]:
import lightgbm as lgb
lgb_clf= lgb.LGBMClassifier(random_state=1)
lgb_clf.fit(x_train, y_train)
new_prediction4= lgb_clf.predict(x_test)

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
n_estimators = [50, 100, 300, 500, 1000]
min_samples_split = [2, 3, 5, 7, 9]
min_samples_leaf = [1, 2, 4, 6, 8]
max_features = ['auto', 'sqrt', 'log2', None]
hyperparameter_grid = {'n_estimators': n_estimators,
'min_samples_leaf': min_samples_leaf,
'min_samples_split': min_samples_split,
'max_features': max_features}
tree_cv= RandomizedSearchCV(tree, param_distributions=hyperparameter_grid, cv=5, random_state=1)
tree_cv.fit(x_train,y_train)

RandomizedSearchCV(cv=5, error_score=nan,
                   estimator=ExtraTreesClassifier(bootstrap=False,
                                                  ccp_alpha=0.0,
                                                  class_weight=None,
                                                  criterion='gini',
                                                  max_depth=None,
                                                  max_features='auto',
                                                  max_leaf_nodes=None,
                                                  max_samples=None,
                                                  min_impurity_decrease=0.0,
                                                  min_impurity_split=None,
                                                  min_samples_leaf=1,
                                                  min_samples_split=2,
                                                  min_weight_fraction_leaf=0.0,
                                        

In [ ]:
new_prediction5= tree_cv.predict(x_test)

In [ ]:
print("Best Parameters: {}".format(tree_cv.best_params_))

Best Parameters: {'n_estimators': 1000, 'min_samples_split': 2, 'min_samples_leaf': 8, 'max_features': None}


In [ ]:
print("Best score is {}".format(tree_cv.best_score_))

Best score is 0.9241249999999999


In [ ]:
from sklearn.metrics import recall_score, accuracy_score, precision_score, f1_score, confusion_matrix
accuracy= accuracy_score(y_true=y_test, y_pred=new_prediction3)
print('Accuracy: {}'.format(round(accuracy*100),2))

Accuracy: 92


In [ ]:
#accuracy using xgboost
accuracy= accuracy_score(y_true=y_test, y_pred=new_prediction3)
round(accuracy,4)

0.9195

In [ ]:
#accuracy using Random forest
accuracy= accuracy_score(y_true=y_test, y_pred=new_prediction)
round(accuracy,4)

0.929

In [ ]:
#accuracy using lightgbm
accuracy= accuracy_score(y_true=y_test, y_pred=new_prediction4)
round(accuracy,4)

0.9375

In [48]:
#accuracy using Extra tree classifier without hyperparameter tuning
accuracy= accuracy_score(y_true=y_test, y_pred=new_prediction2)
round(accuracy,4)

0.928

In [49]:
#accuracy with extra tree classifier with hyperparameter
accuracy= accuracy_score(y_true=y_test, y_pred=new_prediction5)
round(accuracy,4)

0.927